In [1]:
import random, os, tqdm, time, json
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt

import sys
sys.path.append("../../../../")

random.seed(618)
np.random.seed(907)

new_base_path = os.path.join(
    "/Users/minkexiu/Downloads/",
    "/".join(
        os.getcwd().split("/")[-1*(len(sys.path[-1].split("/")) - 1):]
    ),
)
print("storage dir:", new_base_path)
print("code dir:", os.getcwd())

## 创建文件夹。
if not os.path.exists(new_base_path):
    os.makedirs(
        new_base_path
    )
if not os.path.exists(os.path.join(new_base_path, "preprocessedData")):
    os.makedirs(
        os.path.join(new_base_path, "preprocessedData")
    )
if not os.path.exists(os.path.join(new_base_path, "originalData")):
    os.makedirs(
        os.path.join(new_base_path, "originalData")
    )
if not os.path.exists(os.path.join(new_base_path, "trained_models")):
    os.makedirs(
        os.path.join(new_base_path, "trained_models")
    )

def create_originalData_path(filename_or_path):
    return os.path.join(new_base_path, "originalData", filename_or_path)
def create_preprocessedData_path(filename_or_path):
    return os.path.join(new_base_path, "preprocessedData", filename_or_path)
def create_trained_models_path(filename_or_path):
    return os.path.join(new_base_path, "trained_models", filename_or_path)

def millisec2datetime(timestamp):
    time_local = time.localtime(timestamp/1000)
    return time.strftime("%Y-%m-%d %H:%M:%S", time_local)
    
def run_finish():
    # 假设你的字体文件是 'myfont.ttf' 并且位于当前目录下  
    font = FontProperties(fname="/Users/minkexiu/Documents/GitHub/ML_Tryout/SimHei.ttf", size=24)  
    # 创建一个空白的图形  
    fig, ax = plt.subplots()  
    ax.imshow(
        plt.imread("/Users/minkexiu/Downloads/wallhaven-dgxpyg.jpg")
    )
    # 在图形中添加文字  
    ax.text(
        ax.get_xlim()[1] * 0.5, 
        ax.get_ylim()[0] * 0.5, 
        f"程序于这个点跑完：\n{millisec2datetime(time.time()*1000)}", fontproperties=font, ha="center", va="center", color="red"
    )  
    # 设置图形的布局  
    # ax.set_xlim(0, 1)  
    # ax.set_ylim(0, 1)  
    ax.set_xticks([])  
    ax.set_yticks([])  
    ax.patch.set_color("blue")
    # 显示图形  
    plt.show()
        
tqdm.tqdm.pandas() ## 引入这个，就可以在apply的时候用progress_apply了。

import IPython
def kill_current_kernel():
    '''杀死当前的kernel释放内存空间。'''
    IPython.Application.instance().kernel.do_shutdown(True) 
    
def simply_show_data(df1):
    print(df1.shape)
    display(df1.head())
    
def wait_flag(saved_flag_path, time_interval_sec=10):
    print("waiting for", saved_flag_path)
    time_count = 0
    while True:
        if os.path.exists(saved_flag_path):
            break
        time.sleep(time_interval_sec)
        time_count+=time_interval_sec
        print(time_count, end=" ")
    print("finish!!")

class TimerContext:  
    def __enter__(self):  
        self.start_time = str(datetime.now())
        print("start time:", self.start_time)
        return self  
    def __exit__(self, exc_type, exc_val, exc_tb):  
        print("start time:", self.start_time)
        print("end time", str(datetime.now()))

storage dir: /Users/minkexiu/Downloads/ML_Tryout/LLM/20240605_zhongyi/pipline
code dir: /Users/minkexiu/Documents/GitHub/ML_Tryout/LLM/20240605_zhongyi/pipline


https://modelscope.cn/models/iic/nlp_gte_sentence-embedding_chinese-small/summary

In [2]:
from modelscope.models import Model
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

2024-06-05 17:57:07,788 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-06-05 17:57:07,789 - modelscope - INFO - Loading ast index from /Users/minkexiu/.cache/modelscope/ast_indexer
2024-06-05 17:57:07,859 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 2fde54512dc5ba12f922206a0b6b30a3 and a total number of 976 components indexed


In [5]:
pipeline_se = pipeline(
    Tasks.sentence_embedding,
    model=Model.from_pretrained("nlp_gte_sentence-embedding_chinese-small"),
    sequence_length=512
) 

2024-06-05 18:01:17,542 - modelscope - INFO - initialize model from nlp_gte_sentence-embedding_chinese-small
/Users/minkexiu/anaconda3/envs/chattts/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2024-06-05 18:01:17,987 - modelscope - WARNING - No preprocessor field found in cfg.
2024-06-05 18:01:17,987 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-06-05 18:01:17,988 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': 'nlp_gte_sentence-embedding_chinese-small'}. trying to build by task and model information.
2024-06-05 18:0

In [7]:
# # 当输入包含“soure_sentence”与“sentences_to_compare”时，会输出source_sentence中首个句子与sentences_to_compare中每个句子的向量表示，以及source_sentence中首个句子与sentences_to_compare中每个句子的相似度。
# inputs = {
#         "source_sentence": ["吃完海鲜可以喝牛奶吗?"],
#         "sentences_to_compare": [
#             "不可以，早晨喝牛奶不科学",
#             "吃了海鲜后是不能再喝牛奶的，因为牛奶中含得有维生素C，如果海鲜喝牛奶一起服用会对人体造成一定的伤害",
#             "吃海鲜是不能同时喝牛奶吃水果，这个至少间隔6小时以上才可以。",
#             "吃海鲜是不可以吃柠檬的因为其中的维生素C会和海鲜中的矿物质形成砷"
#         ]
#     }
# result = pipeline_se(input=inputs)
# print (result)
# '''
# {'text_embedding': array([[ 1.6415151e-04,  2.2334497e-02, -2.4202393e-02, ...,
#          2.7710509e-02,  2.5980933e-02, -3.1285528e-02],
#        [-9.9107623e-03,  1.3627578e-03, -2.1072682e-02, ...,
#          2.6786461e-02,  3.5029035e-03, -1.5877936e-02],
#        [ 1.9877627e-03,  2.2191243e-02, -2.7656069e-02, ...,
#          2.2540951e-02,  2.1780970e-02, -3.0861111e-02],
#        [ 3.8688166e-05,  1.3409532e-02, -2.9691193e-02, ...,
#          2.9900728e-02,  2.1570563e-02, -2.0719109e-02],
#        [ 1.4484422e-03,  8.5943500e-03, -1.6661938e-02, ...,
#          2.0832840e-02,  2.3828523e-02, -1.1581291e-02]], dtype=float32), 'scores': [0.8859604597091675, 0.9830712080001831, 0.966042160987854, 0.891857922077179]}
# '''
# 当输入仅含有soure_sentence时，会输出source_sentence中每个句子的向量表示。
inputs2 = {
        "source_sentence": [
            "不可以，早晨喝牛奶不科学",
            "吃了海鲜后是不能再喝牛奶的，因为牛奶中含得有维生素C，如果海鲜喝牛奶一起服用会对人体造成一定的伤害",
            "吃海鲜是不能同时喝牛奶吃水果，这个至少间隔6小时以上才可以。",
            "吃海鲜是不可以吃柠檬的因为其中的维生素C会和海鲜中的矿物质形成砷"
        ]
}
result = pipeline_se(input=inputs2)
print (result)

{'text_embedding': array([[-0.04563809, -0.06220782, -0.03775004, ...,  0.01267119,
        -0.01111769, -0.03390383],
       [-0.02073098, -0.04639562, -0.04818704, ..., -0.00754705,
        -0.00731624, -0.02740852],
       [-0.00037597, -0.05922904, -0.0459275 , ..., -0.00697823,
        -0.02154762, -0.02951157],
       [-0.00491675, -0.02552056, -0.03427778, ..., -0.00760836,
        -0.00404084, -0.0509829 ]], dtype=float32), 'scores': []}


In [10]:
# 需在GPU环境运行
# 加载数据集过程可能由于网络原因失败，请尝试重新运行代码
from modelscope.metainfo import Trainers                                                                                                                                                              
from modelscope.msdatasets import MsDataset
from modelscope.trainers import build_trainer

In [11]:
import tempfile
import os
tmp_dir = tempfile.TemporaryDirectory().name
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
print(tmp_dir)

/var/folders/qt/mjqxmxqj2wv8n776nqr_byhh0000gn/T/tmpz09w9c0t


In [12]:
ds = MsDataset.load('dureader-retrieval-ranking', 'zyznull')
train_ds = ds['train'].to_hf_dataset()
dev_ds = ds['dev'].to_hf_dataset()

2024-06-05 18:09:03,059 - modelscope - INFO - dataset_type: 1
/Users/minkexiu/anaconda3/envs/chattts/lib/python3.10/site-packages/datasets/load.py:2524: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/Users/minkexiu/anaconda3/envs/chattts/lib/python3.10/site-packages/datasets/load.py:926: FutureWarning: The repository for dureader-retrieval-ranking contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /Users/minkexiu/.cache/modelscope/hub/datasets/zyznull/dureader-retrieval-ranking/master/meta/dureader-retrieval-ranking.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
huggingfa

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1032a6440>>
Traceback (most recent call last):
  File "/Users/minkexiu/anaconda3/envs/chattts/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:

model_id = 'damo/nlp_gte_sentence-embedding_chinese-base'
def cfg_modify_fn(cfg):
    cfg.task = 'sentence-embedding'
    cfg['preprocessor'] = {'type': 'sentence-embedding','max_length': 256}
    cfg['dataset'] = {
        'train': {
            'type': 'bert',
            'query_sequence': 'query',
            'pos_sequence': 'positive_passages',
            'neg_sequence': 'negative_passages',
            'text_fileds': ['text'],
            'qid_field': 'query_id'
        },
        'val': {
            'type': 'bert',
            'query_sequence': 'query',
            'pos_sequence': 'positive_passages',
            'neg_sequence': 'negative_passages',
            'text_fileds': ['text'],
            'qid_field': 'query_id'
        },
    }
    cfg['train']['neg_samples'] = 4
    cfg['evaluation']['dataloader']['batch_size_per_gpu'] = 30
    cfg.train.max_epochs = 1
    cfg.train.train_batch_size = 4
    return cfg 
kwargs = dict(
    model=model_id,
    train_dataset=train_ds,
    work_dir=tmp_dir,
    eval_dataset=dev_ds,
    cfg_modify_fn=cfg_modify_fn)
trainer = build_trainer(name=Trainers.nlp_sentence_embedding_trainer, default_args=kwargs)
trainer.train()

# 用transformers加载

In [11]:
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

input_texts = [
    "不可以，早晨喝牛奶不科学",
    "吃了海鲜后是不能再喝牛奶的，因为牛奶中含得有维生素C，如果海鲜喝牛奶一起服用会对人体造成一定的伤害",
    "吃海鲜是不能同时喝牛奶吃水果，这个至少间隔6小时以上才可以。",
    "吃海鲜是不可以吃柠檬的因为其中的维生素C会和海鲜中的矿物质形成砷"
]

tokenizer = AutoTokenizer.from_pretrained("nlp_gte_sentence-embedding_chinese-small")
model = AutoModel.from_pretrained("nlp_gte_sentence-embedding_chinese-small", trust_remote_code=True)

batch_dict = tokenizer(input_texts, max_length=8192, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = outputs.last_hidden_state[:, 0]

In [13]:
embeddings.shape

torch.Size([4, 512])

In [14]:
new_texts = [
    '零售木制品、门窗、仪器仪表、实验室专用设备、通讯器材、电子产品、五金交电、制冷专用设备、消防器材、工艺品（不含文物）、文化用品、针纺织品、化工产品（不含危险化学品）、金属材料、体育用品（不含弩）、塑料制品、计算机软件及辅助设备、日用品、服装鞋帽、皮革制品、玩具、化妆品、摄影器材、音响设备、珠宝首饰、宠物用品、建筑材料、玻璃制品、家具；日用电器修理（不符合家用电子电器维修业服务经营规范不得开展经营活动）；技术推广；企业策划；承办展览展示；会议服务；企业管理咨询；翻译服务；组织文化艺术交流活动；电脑图文设计；设计、制作、代理、发布广告；租赁建筑工程机械、建筑工程设备；出租办公用房、商业用房（不得作为有形市场经营用房）；教育咨询（不含培训）；家庭服务（不符合家政服务通用要求不得开展经营活动）；婚庆服务；医学研究（不含诊疗活动）；健康咨询、健康管理（须经审批的诊疗活动除外）；健身服务；修脚服务（不含诊疗活动）；机动车公共停车场经营管理；专业承包；施工总承包；城市园林绿化；物业管理；销售家用电器、机械设备及配件；经济贸易咨询；组装超声波加湿器；汽车租赁（不含九座以上客车）；道路货运代理；零售食品；工程勘察；工程设计。（市场主体依法自主选择经营项目，开展经营活动；依法须经批准的项目，经相关部门批准后依批准的内容开展经营活动；不得从事国家和本区产业政策禁止和限制类项目的经营活动。）',
    "带钢、钢管、型钢、钢材的制造、加工、销售。（依法须经批准的项目，经相关部门批准后方可开展经营活动）",
]